# Assignment 5 - Issue #854

Fernando Mendoza | Adrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

The script should not give an error. Any mistake will be consider as 0.
<br>We want an excel file with all the available presidential elections in this webpage.
https://infogob.jne.gob.pe/Eleccion
<br>The final excel should have these 3 columns 
(Elecciones | ORGANIZACIÓN POLÍTICA | TOTAL VOTOS)

In [26]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install openpyxl

# Run the code line depending on the internet browser you are using (Chrome or Firefox)

## a. Setup for Chrome only

In [3]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd
from openpyxl import load_workbook

# Path for google Chrome
chrome_driver_path = r'chromedriver-win64/chromedriver.exe'
# Configue the service for google chrome
service = Service(executable_path=chrome_driver_path)
# Start the WebDriver for google chrome
driver = webdriver.Chrome(service=service)

## b. Setup for FireFox only

In [36]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time 
import pandas as pd
from openpyxl import load_workbook

# Path for geckodriver (Firefox)
gecko_driver_path = r'geckodriver-v0.34.0-win64\geckodriver.exe'

# Configure the service for Firefox
service = FirefoxService(executable_path=gecko_driver_path)

# Start the WebDriver for Firefox
driver = webdriver.Firefox(service=service)


# 3. Iteration

In [15]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)

In [17]:
dataframes_list = []

url_base = "https://infogob.jne.gob.pe/Eleccion"
# Prepare iteration for all options in "Eleccion"
for number in range(2, 27):
    #First, we go to the website and display dropdown "Tipo de proceso"
    driver.get(url_base)
    proceso = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    proceso.click()
    time.sleep(1)
    # Then select "Elecciones Presidenciales"
    presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidenciales.click()
    time.sleep(1)
    # Then display dropdown "Eleccion"
    eleccion=driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(1)
    # Then select an option in "Eleccion" for the iteration
    election_1 = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(number))
    eleccion_name = election_1.text
    print(f'Nombre de la elección: {eleccion_name}')  # Verificatiión
    election_1.click()
    time.sleep(1)
    # Click in "Ver datos de la eleccion"
    ver_datos = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    ver_datos.click()
    time.sleep(2)
    # Then click on tab "Candidatos y Resultados"
    resultado = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultado.click()
    time.sleep(1)
    # Extract information in table
    table_path = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute('innerHTML')

    # Create headers to avoid issues with read_html
    head = "<table id=\"gridEleccionResultadosElectorales\" class=\"table table-hover table-striped\">"
    table_new = head + table_html[21:-1]
    table = pd.read_html(table_new)
    table_clean = table[0].iloc[:, :]
    table_clean = table_clean.loc[:, ['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    # Insert the election name into the first column
    table_clean.insert(0, 'Elecciones', eleccion_name)
    print(table_clean.head())  # Verificación adicional

    # Add the cleaned DataFrame to the list
    dataframes_list.append(table_clean)

Nombre de la elección: PRESIDENCIAL 2021 - 2DA VUELTA


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22996\1109597775.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_new)


                       Elecciones                 ORGANIZACIÓN POLÍTICA  \
0  PRESIDENCIAL 2021 - 2DA VUELTA  PARTIDO POLITICO NACIONAL PERU LIBRE   
1  PRESIDENCIAL 2021 - 2DA VUELTA                        FUERZA POPULAR   
2  PRESIDENCIAL 2021 - 2DA VUELTA                       VOTOS EN BLANCO   
3  PRESIDENCIAL 2021 - 2DA VUELTA                           VOTOS NULOS   

   TOTAL VOTOS  
0      8836380  
1      8792117  
2       121489  
3      1106816  
Nombre de la elección: PRESIDENCIAL 2021


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22996\1109597775.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_new)


          Elecciones                        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2021         PARTIDO POLITICO NACIONAL PERU LIBRE      2724752
1  PRESIDENCIAL 2021                               FUERZA POPULAR      1930762
2  PRESIDENCIAL 2021                           RENOVACION POPULAR      1692279
3  PRESIDENCIAL 2021  AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL      1674201
4  PRESIDENCIAL 2021                               ACCION POPULAR      1306288
Nombre de la elección: PRESIDENCIAL 2016 - 2DA VUELTA


...

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22996\1109597775.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_new)


          Elecciones   ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1939  CONCENTRACION NACIONAL       262971
1  PRESIDENCIAL 1939       FRENTE PATRIOTICO        76222
Nombre de la elección: PRESIDENCIAL 1936


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22996\1109597775.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_new)


          Elecciones     ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1936  PARTIDO SOCIAL DEMOCRATA        77042
1  PRESIDENCIAL 1936      UNION REVOLUCIONARIA        57838
2  PRESIDENCIAL 1936           FRENTE NACIONAL        46773
3  PRESIDENCIAL 1936       PARTIDO REPUBLICANO        30803
Nombre de la elección: PRESIDENCIAL 1931
          Elecciones       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1931        UNION REVOLUCIONARIA       152149
1  PRESIDENCIAL 1931     PARTIDO APRISTA PERUANO       106088
2  PRESIDENCIAL 1931      PARTIDO DESCENTRALISTA        21950
3  PRESIDENCIAL 1931  PARTIDO COALICION NACIONAL        19640


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22996\1109597775.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(table_new)


In [38]:
# TTransform the list of DataFrames into a single DataFrame
df_vote = pd.concat(dataframes_list, ignore_index=True)

In [40]:
# Save the result and export it to an Excel file with the group name
file_name = 'group_7_ass_5_2024_2.xlsx'
df_vote.to_excel(file_name, index=False)

# Load the Excel file
workbook = load_workbook(file_name)
worksheet = workbook.active

# Adjust the width of the first three columns
for col in ['A', 'B', 'C']:
    worksheet.column_dimensions[col].width = 30

# Save the changes to the Excel file
workbook.save(file_name)